# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [1]:
# Import this library
from nbacktest import Backtest, Strategy

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [2]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.balance/2)/self.prices["AAPL"])
        quantity2 = int((self.balance/2)/self.prices["GOOG"])

        order1 = self.buy("AAPL", quantity1, notes="AAPL buy order")
        order2 = self.buy("GOOG", quantity2, notes="GOOG buy order")

        trade1 = self.create_trade(orders=[order1, order2], notes="Buy and Hold Trade")

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download OHLC data from Yahoo Finance </b>

In [3]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  129.900284  89.141418  130.451959  89.429260  127.713276   
2022-12-27  128.097488  87.275421  129.456984  88.833733  126.806960   
2022-12-28  124.166779  85.816360  129.082614  87.861023  123.999307   
2022-12-29  127.683723  88.287827  128.540789  88.699738  125.831667   
2022-12-30  127.998932  88.069473  128.018639  88.168727  125.536095   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  86.967727  128.974252  86.967727  63814900  17815000  
2022-12-27  86.883365  129.427431  88.645145  69007800  15470900  
2022-12-28  85.727034  127.742826  86.848619  85438400  17879600  
2022-12-29  86.342418  126.087797  86.382121  75703700  18280700  
2022-12-30  86.382124  126.501533  86.714629  77034200  19190300

## <b> Run Backtest </b>

In [4]:
bt = Backtest(data=df_ohlc,
              universe=universe,
              strategy_class=TestStrategy,
              price_column="Close",
              cash=100_000
             )

result = bt.run()

result.head()

Strategy started!
Bought 1239 AAPL and 946 GOOG
Strategy finished!


,BALANCE,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED_TOTAL,EQUITY
ITERATION,,,,
2018-01-02,16.896439,99983.103561,0,100000.000000
2018-01-03,16.896439,100795.052803,0,100811.949242
2018-01-04,16.896439,101211.205696,0,101228.102135
2018-01-05,16.896439,102526.010803,0,102542.907242
2018-01-08,16.896439,102558.536182,0,102575.432621


## <b> Backtest Statistics </b>

In [5]:
print("---------------------------")
print("Final balance: %.2f" % bt.strategy.balance)
print("Final equity: %.2f" % bt.strategy.equity)
print("---------------------------")

---------------------------
Final balance: 16.90
Final equity: 241921.29
---------------------------


In [6]:
bt.get_trade_statistics()

{'n_won': 1,
 'n_lost': 0,
 'n_total': 1,
 'win_rate': 1.0,
 'mean_profit': 141921.29475402832,
 'median_profit': 141921.29475402832,
 'mean_profit_win': 141921.29475402832,
 'mean_profit_loss': nan,
 'median_profit_win': 141921.29475402832,
 'median_profit_loss': nan,
 'std_profit': nan,
 'mad_profit': 0.0,
 'skewness': nan,
 'kurtosis': nan,
 'mean_median_gap': 0.0,
 'mean_profit_over_vol': nan,
 'median_profit_over_vol': nan}

In [7]:
bt.get_equity_statistics_dollar()

{'total_profit': 141921.29475402832,
 'mean_profit': 112.815019677288,
 'median_profit': 256.2007141113281,
 'std_profit': 4002.754169309433,
 'mad_profit': 2424.4150438608067,
 'skewness': -0.19350024200347715,
 'kurtosis': 3.0569111607066723,
 'mean_median_gap': -143.38569443404012,
 'mean_profit_over_vol': 0.028184348802202656,
 'median_profit_over_vol': 0.06400610761353066,
 'max_drawdown': -122839.10111999512}

In [8]:
bt.get_equity_statistics_return()

{'total_return': 1.419212947540283,
 'mean_return': 0.000883801732765282,
 'median_return': 0.0016637553210147038,
 'std_return': 0.019027788238690425,
 'mad_return': 0.014074321392113225,
 'skewness': -0.1545718736379897,
 'kurtosis': 4.071145132942258,
 'mean_median_gap': -0.0007799535882494218,
 'mean_return_over_vol': 0.046447948740999286,
 'median_return_over_vol': 0.0874381877780037,
 'max_drawdown': -0.3432401577912553}

In [9]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,AGE,REASON_CLOSED,POSITIONS_FILLED,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED,POSITIONS_UNFILLED_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE
0,8ab85005-6dd8-4c27-b105-555680262766,OPEN,Buy and Hold Trade,141921.294754,2018-01-02,None,1823 days,None,"{'AAPL': {'quantity': 1239, 'value': 158590.67...",241904.398315,{},0,None,None,None


In [10]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,NOTES,TRADE_ID,ITERATION_REQUESTED,QUANTITY_REQUESTED,PRICE_REQUESTED,FEE_REQUESTED,GROSS_TOTAL_REQUESTED,TOTAL_REQUESTED,ITERATION_FILLED,QUANTITY_FILLED,PRICE_FILLED,FEE_FILLED,GROSS_TOTAL_FILLED,TOTAL_FILLED
0,977bdd66-1e08-466a-956d-0451e98413cf,AAPL,BUY,FILLED,AAPL buy order,8ab85005-6dd8-4c27-b105-555680262766,2018-01-02,1239,40.341896,0.0,49983.609215,49983.609215,2018-01-02,1239,40.341896,0.0,-49983.609215,-49983.609215
1,c181197a-b37f-4dbe-94a6-c4726ffc13f6,GOOG,BUY,FILLED,GOOG buy order,8ab85005-6dd8-4c27-b105-555680262766,2018-01-02,946,52.853588,0.0,49999.494347,49999.494347,2018-01-02,946,52.853588,0.0,-49999.494347,-49999.494347


In [11]:
bt.strategy.positions_filled

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.6766052246)},
 'GOOG': {'quantity': 946, 'value': np.float64(83313.72171020508)}}

In [12]:
bt.strategy.positions_unfilled

{}

In [13]:
bt.strategy.positions_filled

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.6766052246)},
 'GOOG': {'quantity': 946, 'value': np.float64(83313.72171020508)}}